# Lecture 13: Markov chain Monte Carlo

[Markov chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) methods, often abbreviated as MCMC or simply MC, allow us to sample from arbitrary probability distributions. 

The name has two parts: a [Markov chain](https://en.wikipedia.org/wiki/Markov_chain) is a stochastic model that undergoes transitions between different configurations, with the key condition that the probability of any event/transition depends only on the current configuration of the system. And, as you've seen before, [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) methods are ones that make use of random sampling.

MCMC works by constructing a stochastic model -- a Markov chain -- whose stationary distribution is the same as the one that we want to sample from. Monte Carlo simulations of the Markov chain then provide us with samples from the desired distribution.

The Markov chain is defined by a set of transition probabilities $T_{ij}$ between configurations labeled $i$ and $j$. We can ensure that the stationary distribution of the Markov chain is the same as the one that we want to sample by enforcing the [detailed balance](https://en.wikipedia.org/wiki/Detailed_balance) condition

$$
P(i)T_{ij} = P(j)T_{ji}\,,
$$

where $P(i)$ is the probability of configuration $i$. Usually, we aren't able to compute the $P(i)$ exactly, but we can often find their ratios:

$$
\frac{P(i)}{P(j)} = \frac{T_{ji}}{T_{ij}}\,.
$$

If $P$ is a Gibbs distribution, then this condition is

$$
\frac{T_{ij}}{T_{ji}} = \frac{P(j)}{P(i)} = e^{\beta \left(E(i) - E(j)\right)}\,.
$$

Note that there are **many possible ways** to choose the $T_{ij}$ so that the detailed balance condition is satisfied! One common choice is called the [Metropolis rule](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm),

$$
T_{ij} = \begin{cases} 
1 \qquad\qquad\,\, \text{if }E(j) < E(i)\\
e^{-\beta\left(E(j) - E(i)\right)} \;\; \text{if }E(j) > E(i)
\end{cases} \,.
$$


### Example: MCMC sampling for a single spin in an external magnetic field

Let's return to the example of a single spin in an external magnetic field, and let's use MCMC to compute its probability distribution. This is an easy case where we can check our answer analytically. First, let's confirm the analytical result when the interaction energy $\epsilon = 1$ and the temperature $T=1$.

In [2]:
import numpy as np

# Taking our distribution from previous lectures

def gibbs(eps, T):
    """ This function takes the energy eps and temperature T as input
        and returns the Gibbs distribution for a single spin as output """
    
    Z     = np.exp(-eps/T) + np.exp(eps/T)
    p_pos = np.exp( eps/T) / Z
    p_neg = np.exp(-eps/T) / Z
    
    return p_pos, p_neg


# Evaluate the distribution at eps = 1, T = 1

p_pos, p_neg = gibbs(1, 1)

print('At eps = 1, T = 1, the probability of spin up is %lf and spin down is %lf' % (p_pos, p_neg))

At eps = 1, T = 1, the probability of spin up is 0.880797 and spin down is 0.119203


Now we'll write a simulation to *sample* from the Gibbs distribution without computing it explicitly.

The first thing to do is to write down our transition probabilities. There are only two configurations: $\sigma = \pm 1$. With our parameters, the energy of the $\sigma = 1$ configuration is lower than the energy of the $\sigma = -1$ configuration. Thus, the transition probabilities using the Metropolis rule should be

$$
\begin{align}
T_{-+} &= 1\,, \\
T_{+-} &= e^{-\beta\left(E(-1) - E(1)\right)} = e^{-2\beta\epsilon}\,.
\end{align}
$$

To perform the Monte Carlo simulation, we'll start with an arbitrary configuration (say, $\sigma=1$) and simulate many random transitions. We'll also store a record of the configurations as we go along.

In [7]:
import numpy as np
import numpy.random as rng

# Store the configurations

configs = []

# Initialize sigma and transition rates

sigma = 1
T_mp  = 1          # transition from + to - state
T_pm  = np.exp(-2) # transition from - to + state

# Choose the number of steps for our simulation, and go!

n_steps = 10**3

for i in range(n_steps):
    
    # Store the current configuration
    configs.append(sigma)
    
    # Random transition
    if sigma==1:
      r = rng.rand()
      if r<T_pm:
        sigma *= -1

    else:
      sigma *= -1

print('Done.')

Done.


In [5]:
len(configs)

1000000

Now the list `configs` is storing a sample from the probability distribution! Let's check to see how accurate it is. We can compute the probability of the $\sigma=\pm1$ configurations by measuring how often these values appear in `configs`. 

**Note**: we took $10^6$ samples in our simulation. How accurate do you expect the probabilities to be?

In [8]:
# Convert configs into a numpy array so that we can do vector calculations
configs = np.array(configs)

# How often was sigma = 1 in the sample?
p_pos_sample = np.sum(configs==1) / float(len(configs))

# How often was sigma = -1 in the sample?
p_neg_sample = np.sum(configs==-1) / float(len(configs))

# Compare these with the analytical values
print('P(+1) true: %lf\tsample: %lf' % (p_pos, p_pos_sample))
print('P(-1) true: %lf\tsample: %lf' % (p_neg, p_neg_sample))

P(+1) true: 0.880797	sample: 0.875000
P(-1) true: 0.119203	sample: 0.125000
